In [4]:
from unicodedata import category
import pandas as pd

category = pd.read_csv("../data/diner_category_modified.csv")
category.loc[category["diner_category_middle"] == "닭강정"]


,diner_idx,industry_category,diner_category_large,diner_category_middle,diner_category_small,diner_category_detail
318,563309903,음식점,치킨,닭강정,NaN,NaN
438,2031585204,음식점,치킨,닭강정,NaN,NaN
871,202690023,음식점,치킨,닭강정,NaN,NaN
943,869655071,음식점,치킨,닭강정,NaN,NaN
1114,2018141071,음식점,치킨,닭강정,NaN,NaN
...,...,...,...,...,...,...
228373,669696148,음식점,치킨,닭강정,노랑강정,NaN
228579,529000570,음식점,치킨,닭강정,NaN,NaN
230240,1839118381,음식점,치킨,닭강정,천년닭강정,NaN
230499,1628717821,음식점,치킨,닭강정,가마로강정,NaN
